# Built-in Tools in AutoGen Extension

## Overview

AutoGen Extension provides a rich set of pre-built tools that can significantly enhance the capabilities of your agents. These tools cover various functionalities, from interacting with web services to integrating with other AI frameworks.

This notebook will explore some of the key built-in tools available under the `autogen_ext.tools` namespace and demonstrate how to integrate them with your `AssistantAgent`.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai,mcp]>=0.7" rich

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

## HTTP Tools (`autogen_ext.tools.http`)

The `http` module provides tools for making HTTP requests, allowing agents to fetch data from web APIs or interact with web services.

In [2]:
! pip install --quiet -U json-schema-to-pydantic

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool  # ✅ correct import for 0.7.1


# Define an async Python function tool
async def get_public_ip() -> str:
    """Fetches the public IP address."""
    return "203.0.113.45"  # mocked value


# Wrap it as a FunctionTool (no .from_function in 0.7.1)
get_ip_tool = FunctionTool(
    func=get_public_ip,
    description="Get the current public IP address."
)


async def run_http_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

    agent = AssistantAgent(
        name="ip_checker",
        model_client=model_client,
        tools=[get_ip_tool],
        system_message="You are an assistant that can check public IP addresses."
    )

    print("\n🔄 Task: What is my public IP address?")
    print("-" * 40)
    await Console(agent.run_stream(task="What is my public IP address?"))
    await model_client.close()

await run_http_example()


🔄 Task: What is my public IP address?
----------------------------------------
---------- TextMessage (user) ----------
What is my public IP address?
---------- ToolCallRequestEvent (ip_checker) ----------
[FunctionCall(id='call_80YaAQf9BKlGAxHbO9vd2zkz', arguments='{}', name='get_public_ip')]
---------- ToolCallExecutionEvent (ip_checker) ----------
[FunctionExecutionResult(content='203.0.113.45', name='get_public_ip', call_id='call_80YaAQf9BKlGAxHbO9vd2zkz', is_error=False)]
---------- ToolCallSummaryMessage (ip_checker) ----------
203.0.113.45


## MCP Tools (`autogen_ext.tools.mcp`)

The `mcp` module provides tools and workbenches for interacting with Model Context Protocol (MCP) servers. This allows agents to leverage external services and data sources in a standardized, language-agnostic way.

For a more detailed introduction to MCP, refer to `06_MCP_Intro.ipynb` and `07_Workbench.ipynb`.

## Other Built-in Tools

AutoGen Extension offers a variety of other tools for different use cases. You can explore them in the [API documentation](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.html):

- **`autogen_ext.tools.graphrag`**: Tools for interacting with GraphRAG indices.
- **`autogen_ext.tools.langchain`**: Adaptors for using tools defined in LangChain.
- **`autogen_ext.tools.playwright`**: Tools for web automation using Playwright (requires a running Playwright MCP server).

Each of these tools can be integrated with `AssistantAgent` in a similar manner, either directly if they are `FunctionTool` compatible, or via a `Workbench` if they are part of an MCP server.

## Next Steps

This notebook provided a brief overview of some built-in tools. To deepen your understanding:

1.  **Experiment with other tools**: Try integrating `HttpTool` for different API calls or explore `LangChain` adaptors.
2.  **Refer to API Docs**: Consult the official [AutoGen Extension API documentation](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.html) for a complete list and detailed usage of all built-in tools.
3.  **Explore MCP Servers**: Set up and experiment with other MCP servers (e.g., `mcp-server-fetch`, `mcp-server-sqlite`) and integrate their tools via `McpWorkbench`.